In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib import style
style.use('fivethirtyeight')
# more styles can be found here: https://matplotlib.org/3.3.3/gallery/style_sheets/style_sheets_reference.html

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime

# Prepare SQLAlchemy 

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores
query = f'''
    SELECT 
        date,
        AVG(prcp) as avg_prcp
    FROM
        measurement
    WHERE
        date >= (SELECT DATE(MAX(date),'-1 year') FROM measurement)
    GROUP BY
        date
    ORDER BY 
        date
'''

# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_df = pd.read_sql(query, conn)

# Convert the date column to date
prcp_df['date'] = pd.to_datetime(prcp_df['date'])

# Sort the dataframe by date
prcp_df.sort_values('date')

# Set the index to date
prcp_df = prcp_df.set_index('date')

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
prcp_df.plot()
plt.xlabel("Date")
plt.ylabel("Inches")

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?

query = '''
    SELECT
        COUNT(station) AS station_count
    FROM
        station
'''

# Save the query results as a Pandas DataFrame and set the index to the date column
session_df = pd.read_sql(query, conn)

station_count = session_df['station_count'].values[0]

station_count

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.

query = '''
    SELECT 
        s.station AS station_code,
        s.name AS station_name
        ,COUNT(*) AS station_count
    FROM 
        measurement m
        INNER JOIN station s 
        ON m.station = s.station
    GROUP BY
        s.station,
        s.name
    ORDER BY
        station_count DESC
'''

active_stations_df = pd.read_sql(query, conn)

active_stations_df.head()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

active_stations_df.sort_values('station_count', ascending=False, inplace=True)
most_active_session = active_stations_df['station_code'].values[0]
print(f'The most active session is: {most_active_session}')

query = f'''
    SELECT 
        MIN(tobs) AS lowest_temperature,
        MAX(tobs) AS highest_temperature,
        AVG(tobs) AS average_temperature
    FROM
        measurement
    WHERE
        station = '{most_active_session}'
'''

most_active_session_stats_df = pd.read_sql(query, conn)

most_active_session_stats_df

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

query = f'''
    SELECT 
        tobs
    FROM
        measurement
    WHERE
        station = '{most_active_session}'
        AND date >= (SELECT date((SELECT MAX(date) FROM measurement),'-1 year'))
'''

temp_obs_df = pd.read_sql(query, conn)

temp_obs_df.plot.hist(bins=12)

plt.xlabel("Temperature")
plt.tight_layout()

## Part 3 - Data Analysis Assignment

In [ ]:
start_date = datetime.strptime('2012-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2012-01-07', '%Y-%m-%d')

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    query = f'''
    SELECT 
        MIN(tobs),
        MAX(tobs),
        AVG(tobs)
    FROM
        measurement
    WHERE
        date BETWEEN '{start_date}' AND '{end_date}'
    '''

    temp_stats_df = pd.read_sql(query, conn)
    
    temp_stats = temp_stats_df.values[0]
    
    return temp_stats

print(calc_temps(start_date, end_date))

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.

from datetime import datetime
from dateutil.relativedelta import relativedelta

start_date_last_year = start_date + relativedelta(years = -1)
end_date_last_year = end_date + relativedelta(years = -1)

tmin, tmax, tavg = calc_temps(start_date_last_year.strftime("%Y-%m-%d"), end_date_last_year.strftime("%Y-%m-%d"))

print(f'Date range: {start_date_last_year} - {end_date_last_year}')

print(f'''
Min temp: {tmin}
Max temp: {tmax}
Avg Temp: {tavg}
''')

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

fig, ax = plt.subplots()
xpos = 1
yerr = tmax-tmin

bar = ax.bar(xpos, tmax, yerr=yerr, alpha=0.5, color='coral', align="center")
ax.set(xticks=range(xpos), xticklabels="a", title="Trip Avg Temp", ylabel="Temp (F)")
ax.margins(.2, .2)

fig.tight_layout()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

query = f'''
    SELECT 
        s.station,
        s.name,
        s.latitude,
        s.longitude,
        s.elevation
        ,sum(m.prcp) AS total_rainfall
    FROM
        measurement m
        INNER JOIN station s
        ON m.station = s.station
    WHERE
        m.date BETWEEN '{start_date_last_year}' AND '{end_date_last_year}'
    GROUP BY
        s.name
    ORDER BY
        total_rainfall DESC
'''

results_df = pd.read_sql(query, conn)
results_df

In [ ]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    query = f'''
        SELECT 
            *
        FROM
            measurement 
    '''

    results_df = pd.read_sql(query, conn)

    results_df['monthday'] = pd.to_datetime(results_df['date']).dt.strftime('%m-%d')

    filtered_df = results_df.loc[results_df['monthday']==date]

    group_df = filtered_df.groupby('monthday')

    group_agg = group_df.agg({'tobs': ['min', 'max', 'mean']})
    
    daily_normals = list(group_agg.values[0])
    
    daily_normals.insert(0, date)
    
    return daily_normals

daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip
trip_start = '2018-01-01'
trip_end = '2018-01-07'

# Use the start and end date to create a range of dates
trip_dates = pd.date_range(trip_start, trip_end, freq='D')

# Stip off the year and save a list of %m-%d strings
trip_month_day = trip_dates.strftime('%m-%d')

# Loop through the list of %m-%d strings and calculate the normals for each date
normals = []
for date in trip_month_day:
    normals.append(daily_normals(date))
    
normals

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
df = pd.DataFrame(normals, columns=['date', 'tmin', 'tavg', 'tmax'])
df.set_index(['date'],inplace=True)
df.head()

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
df = pd.DataFrame(normals, columns=['date','tmin', 'tavg', 'tmax'])
df.set_index(['date'],inplace=True)
df.head()

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
df.plot(kind='area', stacked=False, x_compat=True, alpha=.2)
plt.tight_layout()
plt.xlabel("Date")
plt.ylabel("Temperature")